<a href="https://colab.research.google.com/github/abbask31/Autism-Facial-Recognition/blob/main/autism_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Pillow==8.3.1
!pip install opencv-python==4.5.3.56
!pip install torch==1.9.0
!pip install torchvision==0.10.0
!pip install kaggle==1.5.12

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached opencv-python-4.5.3.56.tar.gz (89.2 MB)
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Installing build dependencies ... error
error: subprocess-exited-with-error

× pip subprocess to install build dependencies did not run successfully.
│ exit code: 1
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement torch==1.9.0 (from versions: 1.11.0, 1.12.0, 1.12.1, 1.

In [56]:
from google.colab import files

uploaded = files.upload()


Saving kaggle.json to kaggle.json


In [57]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [58]:
!kaggle datasets download -d cihan063/autism-image-data


100% 229M/229M [00:11<00:00, 23.7MB/s]
100% 229M/229M [00:11<00:00, 20.2MB/s]


In [ ]:
import zipfile

with zipfile.ZipFile('autism-image-data.zip', 'r') as zip_ref:
    zip_ref.extractall('autism-image-data')


In [ ]:
import os
import shutil

dataset_dir = '/content/autism-image-data/AutismDataset'
train_dir = os.path.join(dataset_dir, 'train')
test_dir = os.path.join(dataset_dir, 'test')

# Create directories to organize the images
train_autistic_dir = os.path.join(train_dir, 'Autistic')
train_non_autistic_dir = os.path.join(train_dir, 'Non_Autistic')
test_autistic_dir = os.path.join(test_dir, 'Autistic')
test_non_autistic_dir = os.path.join(test_dir, 'Non_Autistic')

os.makedirs(train_autistic_dir, exist_ok=True)
os.makedirs(train_non_autistic_dir, exist_ok=True)
os.makedirs(test_autistic_dir, exist_ok=True)
os.makedirs(test_non_autistic_dir, exist_ok=True)

# Move images to the directories in the train set
for image_name in os.listdir(train_dir):
    src_path = os.path.join(train_dir, image_name)
    if image_name.startswith('Autistic') and not os.path.isdir(src_path):
        dest_path = os.path.join(train_autistic_dir, image_name)
        shutil.move(src_path, dest_path)
        print(f'{image_name} --> {train_autistic_dir}')
    elif image_name.startswith('Non_Autistic') and not os.path.isdir(src_path):
        dest_path = os.path.join(train_non_autistic_dir, image_name)
        shutil.move(src_path, dest_path)
        print(f'{image_name} --> {train_non_autistic_dir}')

# Move images to the directories in the test set
for image_name in os.listdir(test_dir):
    src_path = os.path.join(test_dir, image_name)
    if image_name.startswith('Autistic') and not os.path.isdir(src_path):
        dest_path = os.path.join(test_autistic_dir, image_name)
        shutil.move(src_path, dest_path)
        print(f'{image_name} --> {test_autistic_dir}')
    elif image_name.startswith('Non_Autistic') and not os.path.isdir(src_path):
        dest_path = os.path.join(test_non_autistic_dir, image_name)
        shutil.move(src_path, dest_path)
        print(f'{image_name} --> {test_non_autistic_dir}')


In [ ]:
import matplotlib.pyplot as plt
import cv2
import os
from PIL import Image

def display_images_from_folder(folder_path, num_images):
    images = os.listdir(folder_path)
    num_images = min(num_images, len(images))

    for i in range(num_images):
        image_path = os.path.join(folder_path, images[i])
        image = cv2.imread(image_path)
        
        # Display the image
        plt.figure(figsize=(6, 6))
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.show()



In [ ]:
def resize_images_in_folder(folder_path, target_size):
    images = os.listdir(folder_path)

    for image_name in images:
        image_path = os.path.join(folder_path, image_name)
        image = cv2.imread(image_path)
        
        # Resize the image while maintaining the aspect ratio
        resized_image = cv2.resize(image, target_size)
        
        # Save the resized image back to the same location
        cv2.imwrite(image_path, resized_image)


In [ ]:
# Before resizing
display_images_from_folder(train_autistic_dir, num_images=5)

In [ ]:
# Resize and interpolate all images to 224x224 (standard size for base model)
resize_shape = (224,224)
for dir in os.scandir(dataset_dir):
    # Resize each valid subdir
      resize_images_in_folder(f'{dataset_dir}/{dir.name}/Autistic', resize_shape)
      resize_images_in_folder(f'{dataset_dir}/{dir.name}/Non_Autistic', resize_shape)

In [ ]:
# After resizing
display_images_from_folder(train_autistic_dir, num_images=5)

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

# class RecognitionModel(nn.Module):
#     def __init__(self):
#         super(RecognitionModel, self).__init__()

#         # Load VGG19 model
#         self.base = models.vgg19(pretrained=True)
#         for param in self.base.parameters():
#             param.requires_grad = False
        
#         # Modify the VGG19 classifier
#         self.base.classifier = nn.Identity()  # Remove the default classifier layers
        
#         # Define the remaining layers
#         self.global_max_pooling = nn.AdaptiveMaxPool2d(1)
        
        
#         self.dense1 = nn.Linear(in_features=512, out_features=512)
#         self.dense2 = nn.Linear(in_features=512, out_features=128)
#         self.leaky_relu = nn.LeakyReLU(negative_slope=0.2)
#         self.dropout = nn.Dropout(p=0.2)
#         self.dense3 = nn.Linear(in_features=128, out_features=64)
#         self.dense4 = nn.Linear(in_features=64, out_features=32)
#         self.dense5 = nn.Linear(in_features=32, out_features=2)
#         self.softmax = nn.Softmax(dim=1)
    
    
    
#     def forward(self, x):
#         # Apply the VGG19 base layers
#         x = self.base.features(x)
#         x = self.base.avgpool(x)
#         x = torch.flatten(x, 1)
        
#         # Apply the classifier layers
#         x = self.dense1(x)
#         x = nn.ReLU()(x)
#         x = self.dense2(x)
#         x = nn.ReLU()(x)
#         x = self.dense3(x)
#         x = self.leaky_relu(x)
#         x = self.dropout(x)
#         x = self.dense4(x)
#         x = nn.ReLU()(x)
#         x = self.dense5(x)
#         x = self.softmax(x)
        
#         return x

class RecognitionModel(nn.Module):
    def __init__(self):
        super(RecognitionModel, self).__init__()

        # Load VGG19 model
        self.base = models.vgg19(pretrained=True)
        for param in self.base.parameters():
            param.requires_grad = False
        
        # Modify the VGG19 classifier
        self.base.classifier = nn.Identity()  # Remove the default classifier layers
        
        # Define the remaining layers
        self.layers = nn.Sequential(
            nn.AdaptiveMaxPool2d(1),
            nn.Flatten(),
            nn.Linear(in_features=512, out_features=512),
            nn.ReLU(),
            nn.Linear(in_features=512, out_features=128),
            nn.ReLU(),
            nn.Linear(in_features=128, out_features=64),
            nn.LeakyReLU(negative_slope=0.2),
            nn.Dropout(p=0.2),
            nn.Linear(in_features=64, out_features=32),
            nn.ReLU(),
            nn.Linear(in_features=32, out_features=2),
            nn.Softmax(dim=1)
        )
    
    def forward(self, x):
        # Apply the VGG19 base layers
        x = self.base.features(x)
        x = self.base.avgpool(x)
        
        # Apply the remaining layers
        x = self.layers(x)
        
        return x

In [ ]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# Define the transforms for data preprocessing and augmentation
data_transforms = transforms.Compose([
    # transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),  # Adjust the angle as per your requirement
    transforms.RandomCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    # transforms.RandomApply([transforms.GaussianBlur(3)], p=0.2)
])

valid_dir = os.path.join(dataset_dir, 'valid')

# Create the datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=data_transforms)
test_dataset = datasets.ImageFolder(root=test_dir, transform=data_transforms)
valid_dataset = datasets.ImageFolder(root=valid_dir, transform=data_transforms)

# Create the data loaders
# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)


In [ ]:
# learning_rate = 0.001
learning_rate = 0.0001
num_epochs = 10
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda')
model = RecognitionModel().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
from torch.utils.data import sampler
train_indices = np.arange(len(train_dataset))
best_model = None
best_accuracy = 0
patience = 5  # Number of epochs to wait for improvement
no_improvement_counter = 0

# Initialize the k-fold cross-validation
k = 5
kf = KFold(n_splits=k, shuffle=True)

# Iterate over the k folds
for fold, (train_index, valid_index) in enumerate(kf.split(train_indices)):
    print(f"Fold {fold + 1}:")

    # Create the data samplers
    train_sampler = sampler.SubsetRandomSampler(train_index)
    valid_sampler = sampler.SubsetRandomSampler(valid_index)

    # Create the data loaders for training and validation
    train_loader = DataLoader(train_dataset, batch_size=16, sampler=train_sampler)
    valid_loader = DataLoader(train_dataset, batch_size=16, sampler=valid_sampler)

    for epoch in range(num_epochs):

        model.train()
        print('Began training')
        for images, labels in train_loader:
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Validate the model after each epoch
        model.eval()
        with torch.no_grad():
            total_correct = 0
            total_samples = 0
            for images, labels in valid_loader:
                images = images.to(device)
                labels = labels.to(device)

                # Forward pass
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)

                # Calculate accuracy
                total_samples += labels.size(0)
                total_correct += (predicted == labels).sum().item()

            accuracy = 100 * total_correct / total_samples
            print(f'Epoch [{epoch+1}/{num_epochs}], Validation Accuracy: {accuracy:.2f}%')

            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_model = model.state_dict()
                no_improvement_counter = 0
            else:
                no_improvement_counter += 1
                if no_improvement_counter >= patience:
                    print(f'No improvement in validation accuracy for {patience} epochs. Stopping training.')
                    break

    if no_improvement_counter >= patience:
        break

print('Training finished.')


Fold 1:
Began training
Epoch [1/10], Validation Accuracy: 78.54%
Began training
Epoch [2/10], Validation Accuracy: 76.97%
Began training
Epoch [3/10], Validation Accuracy: 77.56%
Began training
Epoch [4/10], Validation Accuracy: 79.33%
Began training
Epoch [5/10], Validation Accuracy: 77.17%
Began training
Epoch [6/10], Validation Accuracy: 80.51%
Began training
Epoch [7/10], Validation Accuracy: 81.30%
Began training
Epoch [8/10], Validation Accuracy: 80.31%
Began training
Epoch [9/10], Validation Accuracy: 79.33%
Began training
Epoch [10/10], Validation Accuracy: 76.57%
Fold 2:
Began training
Epoch [1/10], Validation Accuracy: 81.69%
Began training
Epoch [2/10], Validation Accuracy: 82.48%
Began training
Epoch [3/10], Validation Accuracy: 80.71%
Began training
Epoch [4/10], Validation Accuracy: 79.53%
Began training
Epoch [5/10], Validation Accuracy: 82.68%
Began training
Epoch [6/10], Validation Accuracy: 78.35%
Began training
Epoch [7/10], Validation Accuracy: 79.53%
Began training

In [ ]:
best_accuracy

In [ ]:
test_acc_history = []
test_loss_history = []

model.load_state_dict(best_model)  # Load the best model state

model.eval()
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        # Calculate accuracy and loss
        total_samples = labels.size(0)
        total_correct = (predicted == labels).sum().item()
        accuracy = 100 * total_correct / total_samples
        loss = criterion(outputs, labels)

        test_acc_history.append(accuracy)
        test_loss_history.append(loss.item())
        
        # Calculate average accuracy
        average_accuracy = sum(test_acc_history) / len(test_acc_history)
        print(f'Average Test Accuracy: {average_accuracy:.2f}%')

# Plot accuracy
plt.plot(test_acc_history, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

# Plot loss
plt.plot(test_loss_history, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()


In [ ]:
torch.save(model.state_dict(), 'model.h5')